# Person Random Walk Publisher

This notebook creates a named person who performs a random walk starting from City Hall.
Location updates (with color) are published to MQTT once per second.

**Usage:**
1. Edit the `PERSON_NAME` and `COLOR` in Cell 2
2. Run all cells to start the walker
3. Launch multiple copies of this notebook with different names to see multiple walkers
4. Use `map_viewer.ipynb` to visualize all walkers on a map

In [ ]:
import asyncio
import json
import math
import random
import time

from simulated_city.config import load_config
from simulated_city.mqtt import MqttConnector, MqttPublisher

In [ ]:
# ===== CONFIGURE YOUR PERSON HERE =====
PERSON_NAME = "Alice"  # Change this for each notebook!
COLOR = "#e74c3c"       # Hex color code (red). Change for different colors.
# ======================================

In [ ]:
# City Hall coordinates (Copenhagen)
CITY_HALL_LNGLAT = (12.5683, 55.6761)

# Random walk parameters
STEP_M = 6.0              # meters per step
STEP_S = 1.0              # seconds between steps
MAX_RADIUS_M = 250.0      # max distance from City Hall
SEED = int(time.time())   # random seed based on current time

# MQTT setup
cfg = load_config()
connector = MqttConnector(cfg.mqtt, client_id_suffix=f"walker-{PERSON_NAME}")
connector.connect()
if not connector.wait_for_connection(timeout=10.0):
    raise RuntimeError("Failed to connect to MQTT broker")

publisher = MqttPublisher(connector)
print(f"✓ Connected to MQTT broker as {PERSON_NAME}")

In [ ]:
async def random_walk_publisher(
    person_name: str,
    color: str,
    *,
    seed: int = 42,
    step_m: float = 6.0,
    step_s: float = 1.0,
    max_radius_m: float = 250.0,
) -> None:
    """
    Perform a random walk and publish location updates to MQTT.
    
    Publishes to topic: persons/{person_name}/location
    Message format: {"lng": float, "lat": float, "color": str, "name": str, "timestamp": float}
    """
    rng = random.Random(seed)
    center_lng, center_lat = CITY_HALL_LNGLAT
    meters_per_deg_lat = 111_320.0
    meters_per_deg_lng = 111_320.0 * math.cos(math.radians(center_lat))
    
    # Start at City Hall
    x_m = 0.0
    y_m = 0.0
    
    topic = f"persons/{person_name}/location"
    print(f"Starting random walk for {person_name} (color: {color})")
    print(f"Publishing to topic: {topic}")
    print(f"Press the stop button (Cell 6) to stop.\n")
    
    step_count = 0
    while True:
        # Random direction
        theta = rng.random() * 2.0 * math.pi
        x_m += step_m * math.cos(theta)
        y_m += step_m * math.sin(theta)
        
        # Keep within max radius (soft boundary)
        r = math.hypot(x_m, y_m)
        if r > max_radius_m:
            scale = max_radius_m / r
            x_m *= scale
            y_m *= scale
        
        # Convert to lng/lat
        lng = center_lng + (x_m / meters_per_deg_lng)
        lat = center_lat + (y_m / meters_per_deg_lat)
        
        # Create message
        message = {
            "lng": lng,
            "lat": lat,
            "color": color,
            "name": person_name,
            "timestamp": time.time(),
        }
        
        # Publish to MQTT
        publisher.publish_json(topic, json.dumps(message), qos=0)
        
        step_count += 1
        if step_count % 10 == 0:
            print(f"  {person_name}: {step_count} steps, at ({lng:.6f}, {lat:.6f})")
        
        await asyncio.sleep(step_s)

In [ ]:
# Start the random walk in the background
task = asyncio.create_task(
    random_walk_publisher(
        PERSON_NAME,
        COLOR,
        seed=SEED,
        step_m=STEP_M,
        step_s=STEP_S,
        max_radius_m=MAX_RADIUS_M,
    )
)
print(f"\n✓ {PERSON_NAME} is walking and publishing to MQTT!")
print("Run the next cell to stop.")

In [ ]:
# Stop the walker
task.cancel()
connector.disconnect()
print(f"✓ {PERSON_NAME} stopped walking.")